In [1]:
import os
from dotenv import load_dotenv

In [2]:
groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)

In [4]:
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x109dcd690>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x109dcc9d0>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, my name is Abishek and i am data scientist")])

AIMessage(content="Hello Abishek!\n\nIt's nice to meet you. That's awesome that you're a data scientist. \n\nWhat kind of work do you do in data science?  Are you working on any interesting projects right now? \n\nI'm always eager to learn more about what people are doing in this field. 😊 \n\n", response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 27, 'total_tokens': 102, 'completion_time': 0.156027436, 'prompt_time': 0.002606972, 'queue_time': None, 'total_time': 0.158634408}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6e15fcd3-fbc5-491c-b5a3-29ed04d66499-0', usage_metadata={'input_tokens': 27, 'output_tokens': 75, 'total_tokens': 102})

In [6]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi, my name is Abishek kumar yadav and I am a data scientist"),
    AIMessage(content="Hello Abishek! \n\nIt's nice to meet you.  \n\nBeing a data scientist is fascinating! What kind of work do you do? What are you most passionate about in the field?  \n\nI'm always eager to learn more about how people use data to make a difference. 😊"),
    HumanMessage(content="hey what's my name and what do i do?")
])

AIMessage(content="You told me your name is Abishek Kumar Yadav and that you are a data scientist!  \n\nIs there anything else you'd like to tell me about yourself or your work? I'm happy to chat. 😊 \n", response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 116, 'total_tokens': 166, 'completion_time': 0.102639857, 'prompt_time': 0.007855055, 'queue_time': None, 'total_time': 0.110494912}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-4931b47a-f6b7-4990-ac16-7405ef0f9bf0-0', usage_metadata={'input_tokens': 116, 'output_tokens': 50, 'total_tokens': 166})

## Message History
-  We can use a message history class to wrap out model and make it statful. This will kep track of input and outputs of the model, and store them in some datastore. Futute interactions will then load those messages and pass them into the cahin as part of the input. Let's see how to use this?

In [15]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


In [16]:
store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()

    return store[session_id]

with_message_history=RunnableWithMessageHistory(model, get_session_history)

In [17]:
config={"configurable":{"session_id":"chat1"}}

In [18]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi, I am abishek kumar and i am a data scientist")],
    config=config
)

In [19]:
response.content

"Hi Abhishek Kumar, it's nice to meet you! \n\nBeing a data scientist is a fascinating field. What kind of work do you specialize in?  \n\nI'm eager to learn more about your experience and projects.  Perhaps you could tell me about:\n\n* **A recent project you're proud of.**\n* **The most challenging problem you've tackled.**\n* **Your favorite tools or techniques in your data science toolbox.** \n\n\nLet's chat! 😄 \n\n"

In [20]:
with_message_history.invoke(
    [HumanMessage(content="What is my name and role i am working?")],
    config=config
)

AIMessage(content="You told me your name is Abhishek Kumar and that you are a data scientist.  \n\nIs there anything else you'd like to tell me about your work or yourself?  I'm here to listen! 😊  \n", response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 151, 'total_tokens': 199, 'completion_time': 0.098175283, 'prompt_time': 0.008186366, 'queue_time': None, 'total_time': 0.106361649}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-de996763-a469-401a-b6e0-ca47539847b6-0', usage_metadata={'input_tokens': 151, 'output_tokens': 48, 'total_tokens': 199})

In [21]:
## change the session Id
config2={"configurable":{"session_id":"chat2"}}

In [22]:
config2

{'configurable': {'session_id': 'chat2'}}

In [23]:
response=with_message_history.invoke(
    [HumanMessage(content="What my name?")],
    config=config
)

In [24]:
response.content

'Your name is Abhishek Kumar.  \n\nI remember that from our earlier conversation!  😊  \n\nIs there anything else I can help you with?\n'

In [25]:
response=with_message_history.invoke(
    [HumanMessage(content="My name is Abishek?")],
    config=config
)

In [26]:
response.content

"You're right!  My apologies, I seem to have made a small mistake. \n\nYour name is Abishek, not Abhishek. I'm still under development and learning to be more accurate. Thanks for pointing that out!  \n\nWhat can I help you with now? \n"

In [27]:
response=with_message_history.invoke(
    [HumanMessage(content="What my name?")],
    config=config2
)

In [28]:
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it! 😊\n"

In [29]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, I am abishek kumar and i am a data scientist'), AIMessage(content="Hi Abhishek Kumar, it's nice to meet you! \n\nBeing a data scientist is a fascinating field. What kind of work do you specialize in?  \n\nI'm eager to learn more about your experience and projects.  Perhaps you could tell me about:\n\n* **A recent project you're proud of.**\n* **The most challenging problem you've tackled.**\n* **Your favorite tools or techniques in your data science toolbox.** \n\n\nLet's chat! 😄 \n\n", response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 28, 'total_tokens': 134, 'completion_time': 0.219783684, 'prompt_time': 0.002621402, 'queue_time': None, 'total_time': 0.222405086}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bc06071f-b002-4955-8aab-695c81531582-0', usage_metadata={'input_tokens': 28, 'output_tokens': 106, 'tot

In [30]:
ChatMessageHistory

langchain_core.chat_history.InMemoryChatMessageHistory

## Prompt Template
- Prompt Templates help to turn user information into a format that the LLm can work with. In this case, the raw user input is hist a message, which are passing to the llm. Lets now make that a bit more complicated. First, Let's add in a system message with some custom intstructions (but still taking message as input) Next, we'll ass in more input besides just the message.

In [31]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [32]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistance. Answer all the question to the nest of yout abililty"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [33]:
chain=prompt|model

In [34]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is Abishek")]})

AIMessage(content="Hi Abishek, it's nice to meet you! 👋\n\nHow can I help you today? 😊 \n\n", response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 40, 'total_tokens': 68, 'completion_time': 0.056959223, 'prompt_time': 0.002913242, 'queue_time': None, 'total_time': 0.059872465}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c36d9a62-632b-4c39-af2e-13665116634c-0', usage_metadata={'input_tokens': 40, 'output_tokens': 28, 'total_tokens': 68})

In [35]:
with_message_history=RunnableWithMessageHistory(chain, get_session_history)

In [36]:
config={"configurable":{"session_id":"chat3"}}
with_message_history.invoke(
    [HumanMessage(content="Hi my name is Abishek")],
    config=config
)

AIMessage(content="Hello Abishek, it's nice to meet you! 👋\n\nIs there anything I can help you with today? 😄  I'm ready for your questions!\n", response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 40, 'total_tokens': 78, 'completion_time': 0.078062678, 'prompt_time': 0.002905632, 'queue_time': None, 'total_time': 0.08096831}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-129f003c-54cc-4e78-a390-0dbfeca500fd-0', usage_metadata={'input_tokens': 40, 'output_tokens': 38, 'total_tokens': 78})

In [37]:
## Add more complexity

prompt= ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistance. Answer all question to the best of your ability in language {language}"
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain=prompt|model

In [38]:
response=chain.invoke({"messages":[HumanMessage(content="Hi my name is Abishek")], "language":"Nepali"})

In [39]:
response.content

'नमस्ते Abishek! 😊  खुशी हो तपाईंले यहाँ आए । म सधैँ तपाईंको लागि मदत गर्न तयार छु। के म तपाईंलाई केही गर्न सक्छु? \n\n'

- Lets now wrap this more complicated chain in a message history class. This time, because there are multiple keys in the input, we need to specity the correct key to use to save the chat history.

In [40]:
with_message_history=RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="messages"
)

In [41]:
config={"configurable":{"session_id":"chat4"}}

In [42]:
response=with_message_history.invoke(
    {'messages':[HumanMessage(content="Hi, I an Abishek")], "language":"Nepali"},
    config=config
)

In [43]:
response.content

'Namaste Abhishek! 😊 \n\nमलाई तपाईंको परिचय राम्रो लाग्यो। कसरी छुँ तपाईं ? \n'

In [44]:
response=with_message_history.invoke(
    {'messages':[HumanMessage(content="Whats my name")], "language":"Nepali"},
    config=config
)

In [45]:
response.content

'हाम्रो कुराकानीमा तपाईंले आफ्नो नाम Abishek भन्नुभयो।  \n'

## Managing the conversation History
- One importanct concept to understand when building chatbots is how to manage converation history. If left unmanged, the list of message will grow unbounded and potentially overflow the context window of the LLM. Therefore. It is important to add a step that limits the size of the message you passing in.

"Trim Message" helper to redue how many messages we're sending to the model. The trimer allows us to speciy how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages.

In [46]:
from langchain_core.messages import SystemMessage, trim_messages

In [52]:
trimmer=trim_messages(
    max_tokens=75, 
    strategy="last", # It going to focus on the last conversation that it had.
    token_counter=model, # Function or llm counting tokens in Bese message list.
    include_system=True, # Include system message
    allow_partial=False,
    start_on="human",
)

In [53]:
messages=[
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="Hi! I'm Bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="Whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="Thanks"),
    AIMessage(content="No, problem"),
    HumanMessage(content="Having fun?"),
    AIMessage(content="Yes!")
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content="Hi! I'm Bob"),
 AIMessage(content='hi!'),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='Whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='Thanks'),
 AIMessage(content='No, problem'),
 HumanMessage(content='Having fun?'),
 AIMessage(content='Yes!')]

In [54]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

In [57]:
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)
chain.invoke(
    {
    "messages": messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)

AIMessage(content='You said you like vanilla ice cream! 😊  \n', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 125, 'total_tokens': 138, 'completion_time': 0.026, 'prompt_time': 0.006419434, 'queue_time': None, 'total_time': 0.032419434}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-38d3e3be-09a5-4bdc-9145-c28be475619e-0', usage_metadata={'input_tokens': 125, 'output_tokens': 13, 'total_tokens': 138})

In [58]:
## Lets wrap thus in the messages History.
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",

)
config={"configurable":{"session_id":"chat5"}}

In [62]:
with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="My name is Abishek")],
        "language":"English",
    },
    config=config,
)

AIMessage(content="Hi Abishek! It's nice to meet you. 😊  \n\nHow are you doing today?\n", response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 125, 'total_tokens': 150, 'completion_time': 0.05, 'prompt_time': 0.007621135, 'queue_time': None, 'total_time': 0.057621135000000004}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-151ab2c1-78ac-4d48-b742-ec2d7b0f0d04-0', usage_metadata={'input_tokens': 125, 'output_tokens': 25, 'total_tokens': 150})

In [63]:
with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name")],
        "language":"English",
    },
    config=config,
)

AIMessage(content='Your name is Bob. 😊  \n\n\n\n', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 122, 'total_tokens': 132, 'completion_time': 0.02, 'prompt_time': 0.009054717, 'queue_time': None, 'total_time': 0.029054717}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-73169760-ad4f-4268-a2c6-4405314f27e9-0', usage_metadata={'input_tokens': 122, 'output_tokens': 10, 'total_tokens': 132})